In [1]:
import subprocess
import os
from langchain_community.graphs import Neo4jGraph
import re

In [2]:
graph = Neo4jGraph(
    url="neo4j+s://46a80122.databases.neo4j.io:7687",
    username="<user>", 
    password="<pass>",
    refresh_schema=True, 
)

ValueError: Cannot resolve address 46a80122.databases.neo4j.io:7687

In [105]:
schema=graph.schema
print(graph.schema)

Node properties are the following:
Article {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},Code {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},Dataset {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},SoftwareApplication {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING}
Relationship properties are the following:

The relationships are the following:
(:Article)-[:uses]->(:Dataset),(:Article)-[:provides]->(:Code),(:Article)-[:provides]->(:SoftwareApplication),(:Article)-[:cites]->(:Article)


In [106]:
question = "How many articles uses a dataset?"

In [107]:
full_prompt = f"<|im_start|>system\nCreate a Cypher statement for the graph with following schema:\n{schema}\n\nTo answer the following question:<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"

In [108]:
full_prompt

'<|im_start|>system\nCreate a Cypher statement for the graph with following schema:\nNode properties are the following:\nArticle {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},Code {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},Dataset {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},SoftwareApplication {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Article)-[:uses]->(:Dataset),(:Article)-[:provides]->(:Code),(:Article)-[:provides]->(:SoftwareApplication),(:Article)-[:cites]->(:Article)\n\nTo answer the following question:<|im_end|>\n<|im_start|>user\nHow many articles uses a dataset?<|im_end|>\n<|im_start|>assistant\n'

In [109]:
model_path = "/Users/dhruvchandel/Thesis_Git/NLToSPARQL/models/NLToCypher_Neo4j_3b.Q4.gguf"  
llama_cli_path = "/Users/dhruvchandel/LLaMaCPP/llama.cpp/llama-cli" 

In [117]:
command = f"{llama_cli_path} -m {model_path} -p '{full_prompt}'  --n-predict 180 --temp 0.2 --top_p 0.9"

In [119]:

result = subprocess.run(command, shell=True, capture_output=True, text=True)
print(type(result))
print("Output:", result.stdout)
print("Error:", result.stderr)


<class 'subprocess.CompletedProcess'>
Output: 
Error: error while handling argument "--logit-bias": invalid input format

usage:
-l,    --logit-bias TOKEN_ID(+/-)BIAS   modifies the likelihood of token appearing in the completion,
                                        i.e. `--logit-bias 15043+1` to increase likelihood of token ' Hello',
                                        or `--logit-bias 15043-1` to decrease likelihood of token ' Hello'


to show complete usage, run with -h



In [112]:
def extract_cypher_query(text):
    patterns = [
        r"'[\s\n]*(MATCH[\s\S]*?)'",
        r"'''[\s\n]*(MATCH[\s\S]*?)'''",
        r"`[\s\n]*(MATCH[\s\S]*?)`",
        r"```[\s\n]*(MATCH[\s\S]*?)```",
        r"```cypher[\s\n]*(MATCH[\s\S]*?)```"  # New pattern for ```cypher ... ``` format
    ]
    
    for pattern in patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            return matches[0].strip()
    
    # If no matches found, try to find any Cypher-like query
    cypher_pattern = r"MATCH[\s\S]*?RETURN.*"
    match = re.search(cypher_pattern, text, re.IGNORECASE)
    if match:
        return match.group(0).strip()
    
    return None

In [113]:
cypher_query = extract_cypher_query(result.stdout)

In [114]:
print(cypher_query)

MATCH (a:Article)-[:uses]->(d:Dataset)
RETURN count(a)


In [115]:
def execute_cypher_query(graph, cypher_query):
    with graph._driver.session() as session:  # Use the session from the existing graph object
        result = session.run(cypher_query)
        return [record.data() for record in result]

In [116]:
print(execute_cypher_query(graph, cypher_query))

[{'count(a)': 29}]
